In [ ]:
!uv pip install matplotlib seaborn scikit-learn

# Diamond Price Analysis and Prediction

This notebook analyzes diamond prices across multiple dimensions and builds a predictive model for diamond pricing.

## Dataset Overview
- Dataset: Seaborn diamonds dataset (~54,000 diamonds)
- Features: carat, cut, color, clarity, depth, table, price, x, y, z
- Target: price (in US dollars)

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from snowflake.snowpark.context import get_active_session
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
print('Initialized')

In [ ]:
SELECT * FROM VNEXT.PUBLIC.DIAMONDS

In [2]:
df = pd.read_csv('diamonds.csv') # Note: We're worrking on hooking up cell references 

#df = session.table("VNEXT.PUBLIC.DIAMONDS").to_pandas()
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
df.head()

In [3]:
print("Dataset Info:")
df.info()
print("\nBasic Statistics:")
df.describe()

In [4]:
print("Missing values:")
print(df.isnull().sum())
print("\nUnique values in categorical columns:")
for col in ['cut', 'color', 'clarity']:
    print(f"{col}: {df[col].unique()}")

## 2. Exploratory Data Analysis

### 2.1 Price Distribution

In [5]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].hist(df['price'], bins=50, alpha=0.7)
axes[0].set_title('Distribution of Diamond Prices')
axes[0].set_xlabel('Price ($)')
axes[0].set_ylabel('Frequency')

axes[1].hist(np.log(df['price']), bins=50, alpha=0.7)
axes[1].set_title('Distribution of Log(Price)')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"Price statistics:")
print(f"Mean: ${df['price'].mean():.2f}")
print(f"Median: ${df['price'].median():.2f}")
print(f"Min: ${df['price'].min():.2f}")
print(f"Max: ${df['price'].max():.2f}")

### 2.2 Analysis by Cut Quality

In [6]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

sns.boxplot(data=df, x='cut', y='price', ax=axes[0, 0])
axes[0, 0].set_title('Price Distribution by Cut')
axes[0, 0].tick_params(axis='x', rotation=45)

cut_counts = df['cut'].value_counts()
axes[0, 1].bar(cut_counts.index, cut_counts.values)
axes[0, 1].set_title('Count of Diamonds by Cut')
axes[0, 1].tick_params(axis='x', rotation=45)

cut_avg_price = df.groupby('cut')['price'].mean().sort_values(ascending=False)
axes[1, 0].bar(cut_avg_price.index, cut_avg_price.values)
axes[1, 0].set_title('Average Price by Cut')
axes[1, 0].tick_params(axis='x', rotation=45)

sns.violinplot(data=df, x='cut', y='price', ax=axes[1, 1])
axes[1, 1].set_title('Price Distribution Density by Cut')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Average price by cut:")
print(cut_avg_price)

### 2.3 Analysis by Color

In [7]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

sns.boxplot(data=df, x='color', y='price', ax=axes[0, 0])
axes[0, 0].set_title('Price Distribution by Color')

color_counts = df['color'].value_counts()
axes[0, 1].bar(color_counts.index, color_counts.values)
axes[0, 1].set_title('Count of Diamonds by Color')

color_avg_price = df.groupby('color')['price'].mean().sort_values(ascending=False)
axes[1, 0].bar(color_avg_price.index, color_avg_price.values)
axes[1, 0].set_title('Average Price by Color')

sns.violinplot(data=df, x='color', y='price', ax=axes[1, 1])
axes[1, 1].set_title('Price Distribution Density by Color')

plt.tight_layout()
plt.show()

print("Average price by color:")
print(color_avg_price)

### 2.4 Analysis by Clarity

In [8]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

sns.boxplot(data=df, x='clarity', y='price', ax=axes[0, 0])
axes[0, 0].set_title('Price Distribution by Clarity')
axes[0, 0].tick_params(axis='x', rotation=45)

clarity_counts = df['clarity'].value_counts()
axes[0, 1].bar(clarity_counts.index, clarity_counts.values)
axes[0, 1].set_title('Count of Diamonds by Clarity')
axes[0, 1].tick_params(axis='x', rotation=45)

clarity_avg_price = df.groupby('clarity')['price'].mean().sort_values(ascending=False)
axes[1, 0].bar(clarity_avg_price.index, clarity_avg_price.values)
axes[1, 0].set_title('Average Price by Clarity')
axes[1, 0].tick_params(axis='x', rotation=45)

sns.violinplot(data=df, x='clarity', y='price', ax=axes[1, 1])
axes[1, 1].set_title('Price Distribution Density by Clarity')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Average price by clarity:")
print(clarity_avg_price)

### 2.5 Analysis by Carat Weight

In [9]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].scatter(df['carat'], df['price'], alpha=0.5)
axes[0, 0].set_title('Price vs Carat Weight')
axes[0, 0].set_xlabel('Carat')
axes[0, 0].set_ylabel('Price ($)')

axes[0, 1].hist(df['carat'], bins=50, alpha=0.7)
axes[0, 1].set_title('Distribution of Carat Weights')
axes[0, 1].set_xlabel('Carat')
axes[0, 1].set_ylabel('Frequency')

df['carat_binned'] = pd.cut(df['carat'], bins=10)
carat_price = df.groupby('carat_binned')['price'].mean()
axes[1, 0].bar(range(len(carat_price)), carat_price.values)
axes[1, 0].set_title('Average Price by Carat Range')
axes[1, 0].set_xlabel('Carat Range')
axes[1, 0].set_ylabel('Average Price ($)')
axes[1, 0].tick_params(axis='x', rotation=45)

axes[1, 1].scatter(df['carat'], np.log(df['price']), alpha=0.5)
axes[1, 1].set_title('Log(Price) vs Carat Weight')
axes[1, 1].set_xlabel('Carat')
axes[1, 1].set_ylabel('Log(Price)')

plt.tight_layout()
plt.show()

correlation = df['carat'].corr(df['price'])
print(f"Correlation between carat and price: {correlation:.3f}")

### 2.6 Diamond Dimensions Analysis

In [10]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

for i, dim in enumerate(['x', 'y', 'z']):
    axes[0, i].scatter(df[dim], df['price'], alpha=0.5)
    axes[0, i].set_title(f'Price vs {dim.upper()} dimension')
    axes[0, i].set_xlabel(f'{dim.upper()} (mm)')
    axes[0, i].set_ylabel('Price ($)')
    
    axes[1, i].hist(df[dim], bins=50, alpha=0.7)
    axes[1, i].set_title(f'Distribution of {dim.upper()} dimension')
    axes[1, i].set_xlabel(f'{dim.upper()} (mm)')
    axes[1, i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("Correlations with price:")
for dim in ['x', 'y', 'z']:
    corr = df[dim].corr(df['price'])
    print(f"{dim}: {corr:.3f}")

### 2.7 Correlation Matrix

In [11]:
numeric_cols = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix of Numeric Features')
plt.show()

print("Strongest correlations with price:")
price_corr = correlation_matrix['price'].abs().sort_values(ascending=False)
print(price_corr[1:])

### 2.8 Multi-dimensional Analysis

In [12]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

sns.scatterplot(data=df.sample(5000), x='carat', y='price', hue='cut', ax=axes[0, 0])
axes[0, 0].set_title('Price vs Carat by Cut')

sns.scatterplot(data=df.sample(5000), x='carat', y='price', hue='color', ax=axes[0, 1])
axes[0, 1].set_title('Price vs Carat by Color')

sns.scatterplot(data=df.sample(5000), x='carat', y='price', hue='clarity', ax=axes[1, 0])
axes[1, 0].set_title('Price vs Carat by Clarity')

price_by_features = df.groupby(['cut', 'color'])['price'].mean().unstack()
sns.heatmap(price_by_features, annot=True, fmt='.0f', cmap='viridis', ax=axes[1, 1])
axes[1, 1].set_title('Average Price by Cut and Color')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing for Machine Learning

In [13]:
df_ml = df.copy()

df_ml['volume'] = df_ml['x'] * df_ml['y'] * df_ml['z']
df_ml['surface_area'] = 2 * (df_ml['x']*df_ml['y'] + df_ml['x']*df_ml['z'] + df_ml['y']*df_ml['z'])
df_ml['ratio_xy'] = df_ml['x'] / df_ml['y']
df_ml['ratio_xz'] = df_ml['x'] / df_ml['z']

df_ml = df_ml.replace([np.inf, -np.inf], np.nan).dropna()

label_encoders = {}
for col in ['cut', 'color', 'clarity']:
    le = LabelEncoder()
    df_ml[col + '_encoded'] = le.fit_transform(df_ml[col])
    label_encoders[col] = le

feature_cols = ['carat', 'depth', 'table', 'x', 'y', 'z', 'volume', 'surface_area', 
                'ratio_xy', 'ratio_xz', 'cut_encoded', 'color_encoded', 'clarity_encoded']

X = df_ml[feature_cols]
y = df_ml['price']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Features: {feature_cols}")

## 4. Model Building and Training

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }
    
    print(f"{name} Results:")
    print(f"  RMSE: ${rmse:.2f}")
    print(f"  MAE: ${mae:.2f}")
    print(f"  R²: {r2:.4f}")
    print()

## 5. Model Evaluation and Comparison

In [15]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

for i, (name, result) in enumerate(results.items()):
    y_pred = result['predictions']
    
    axes[i, 0].scatter(y_test, y_pred, alpha=0.5)
    axes[i, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[i, 0].set_title(f'{name}: Predicted vs Actual')
    axes[i, 0].set_xlabel('Actual Price ($)')
    axes[i, 0].set_ylabel('Predicted Price ($)')
    
    residuals = y_test - y_pred
    axes[i, 1].scatter(y_pred, residuals, alpha=0.5)
    axes[i, 1].axhline(y=0, color='r', linestyle='--')
    axes[i, 1].set_title(f'{name}: Residual Plot')
    axes[i, 1].set_xlabel('Predicted Price ($)')
    axes[i, 1].set_ylabel('Residuals ($)')

plt.tight_layout()
plt.show()

comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[name]['rmse'] for name in results.keys()],
    'MAE': [results[name]['mae'] for name in results.keys()],
    'R²': [results[name]['r2'] for name in results.keys()]
})

print("Model Comparison:")
print(comparison_df)

## 6. Feature Importance Analysis

In [16]:
best_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10))

## 7. Price Prediction Examples

In [17]:
def predict_diamond_price(carat, cut, color, clarity, depth, table, x, y, z):
    
    volume = x * y * z
    surface_area = 2 * (x*y + x*z + y*z)
    ratio_xy = x / y if y != 0 else 0
    ratio_xz = x / z if z != 0 else 0
    
    cut_encoded = label_encoders['cut'].transform([cut])[0]
    color_encoded = label_encoders['color'].transform([color])[0]
    clarity_encoded = label_encoders['clarity'].transform([clarity])[0]
    
    features = np.array([[carat, depth, table, x, y, z, volume, surface_area, 
                         ratio_xy, ratio_xz, cut_encoded, color_encoded, clarity_encoded]])
    
    prediction = best_model.predict(features)[0]
    return prediction

sample_diamonds = [
    {'carat': 1.0, 'cut': 'Ideal', 'color': 'G', 'clarity': 'VS1', 'depth': 62.0, 'table': 57.0, 'x': 6.3, 'y': 6.3, 'z': 3.9},
    {'carat': 2.0, 'cut': 'Premium', 'color': 'F', 'clarity': 'VVS2', 'depth': 61.0, 'table': 58.0, 'x': 8.1, 'y': 8.1, 'z': 5.0},
    {'carat': 0.5, 'cut': 'Good', 'color': 'J', 'clarity': 'SI1', 'depth': 63.0, 'table': 60.0, 'x': 5.0, 'y': 5.0, 'z': 3.2}
]

print("Diamond Price Predictions:")
print("=" * 50)

for i, diamond in enumerate(sample_diamonds, 1):
    predicted_price = predict_diamond_price(**diamond)
    print(f"\nDiamond {i}:")
    print(f"  Specifications: {diamond['carat']} carat, {diamond['cut']}, {diamond['color']}, {diamond['clarity']}")
    print(f"  Predicted Price: ${predicted_price:.2f}")

## 8. Summary and Insights

### Key Findings:

1. **Carat Weight**: The strongest predictor of diamond price, showing a strong positive correlation

2. **Physical Dimensions**: X, Y, Z dimensions are highly correlated with both carat weight and price

3. **Cut Quality**: Surprisingly, 'Ideal' cuts don't always command the highest average prices, possibly due to market dynamics

4. **Color**: Near-colorless diamonds (D, E, F) generally command higher prices

5. **Clarity**: Higher clarity grades (FL, IF, VVS) typically have higher prices, but the relationship is complex

### Model Performance:
- Random Forest performs better than Linear Regression for this dataset
- The model achieves good predictive accuracy for diamond pricing
- Feature engineering (volume, surface area, ratios) provides additional predictive power

### Business Applications:
- Price estimation for diamond inventory
- Market analysis and pricing strategy
- Quality assessment and valuation
- Investment and trading decisions